In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import logging
import time

from scipy.ndimage import zoom
from giza_actions.action import action
from giza_actions.task import task
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ModuleNotFoundError: No module named 'sklearn'

In [3]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
batch_size = 64

Define the model


In [5]:
input_size = 99
output_size = 1

# Define hidden layers sizes and dropout rates
hidden_size = [256, 128, 64]
dropout_rates = [0.1, 0.1]
learning_rate = 0.001
num_epochs = 100

criterion = nn.MSELoss()

In [ ]:
class RegressionNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers_sizes=[256, 128, 64], dropout_rates=[0.1, 0.1]):
        super(RegressionNN, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        self.dropouts = nn.ModuleList()

        layer_sizes = [input_size] + hidden_layers_sizes
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            self.batch_norms.append(nn.BatchNorm1d(layer_sizes[i+1]))
            self.dropouts.append(nn.Dropout(dropout_rates[min(i, len(dropout_rates)-1)]))
        
        self.output_layer = nn.Linear(hidden_layers_sizes[-1], output_size)
        
    def forward(self, x):
        for i in range(len(self.layers)):
            x = self.layers[i](x)
            x = self.batch_norms[i](x)
            x = nn.ReLU()(x)
            x = self.dropouts[i](x)
            
        x = self.output_layer(x)
        return x


In [ ]:
@task(name=f'Prepare Datasets')
def prepare_datasets(X, y):
    print("Prepare dataset...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
    test_dataset = torchvision.datasets.MNIST(root='./data', train=False)

    X_train_tensor = torch.tensor(X_train.values.astype(np.float32))
    y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).view(-1, 1)
    X_test_tensor = torch.tensor(X_test.values.astype(np.float32))
    y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).view(-1, 1)

    print("✅ Datasets prepared successfully")

    return X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor

In [ ]:
@task(name=f'Create Loaders')
def create_data_loaders(x_train, y_train, x_test, y_test):
    print("Create loaders...")

    train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

    print("✅ Loaders created!")

    return train_loader, test_loader

In [ ]:
@task(name=f'Train model')
def train_model(train_loader):
    print("Train model...")

    model = RegressionNN(input_size, hidden_size, output_size=1).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device).reshape(-1, 14*14)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    print("✅ Model trained successfully")
    return model

In [ ]:
@task(name=f'Test model')
def test_model(model, test_loader):
    print("Test model...")
    # with torch.no_grad():
    #     n_correct = 0
    #     n_samples = 0
    #     for inputs, labels in test_loader:
    #         inputs = inputs.to(device).reshape(-1, 14*14)
    #         labels = labels.to(device)
    #         outputs = model(inputs)
    #         _, predicted = torch.max(outputs.data, 1)
    #         n_samples += labels.size(0)
    #         n_correct += (predicted == labels).sum().item()
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
            

In [ ]:
@action(name=f'Execution', log_prints=True )
def execution():
    x_train, y_train, x_test, y_test = prepare_datasets()
    train_loader, test_loader = create_data_loaders(
        x_train, y_train, x_test, y_test)
    model = train_model(train_loader)
    test_model(model, test_loader)
